In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import matplotlib.pyplot as plt
import cv2

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [6]:
import cv2
import tensorflow as tf
import numpy as np

# Load the pre-trained human detection classifier (Haar cascade)
classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained human classification model
classification_model = tf.keras.applications.ResNet50(weights='imagenet')

# Initialize the video capture object
video_capture = cv2.VideoCapture(0)  # 0 refers to the default camera

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    # Convert the frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect human faces in the frame
    humans = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in humans:
        # Extract the region of interest (ROI) containing the detected human
        roi = frame[y:y+h, x:x+w]
        
        # Preprocess the ROI for classification
        roi = cv2.resize(roi, (224, 224))
        roi = tf.keras.applications.resnet.preprocess_input(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # Perform human classification
        predictions = classification_model.predict(roi)
        
        # Get the human probability of the predicted class
        human_probability = predictions[0][np.argmax(predictions)]
        
        # Draw bounding boxes and display the classification result
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'Human: {human_probability:.2f}%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Display the resulting frame with detected humans and classification results
    cv2.imshow('Human Detection and Classification', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 108ms/step
